# 3 - Genome Wide Association Study

In [2]:
import os, sys
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

## GWAS using DATScan data

First of all we need to load the file related to the DATScan information.

In [21]:
DATScan = pd.read_csv("../Imaging/DATScan_Norm.csv")
images = DATScan.iloc[:,np.r_[0,3:7]]
images = images.groupby('PATNO').first().reset_index()
images

,PATNO,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,2.99,3.43,2.328620,1.973532
1,3001,1.56,1.12,-0.246956,-0.796382
2,3002,2.13,2.28,0.255480,0.339528
3,3003,2.19,1.27,-0.118658,-0.401457
4,3004,5.09,5.30,2.949289,2.400680
...,...,...,...,...,...
990,75484,1.35,1.24,-0.765017,-1.158726
991,75505,1.90,1.66,0.065569,-0.486548
992,75524,1.36,1.76,-0.665445,0.269728
993,75537,1.46,1.03,0.065569,-1.070600


In [48]:
ppmi_info = pd.read_csv('../PPMI_data/Info files/Screening___Demographics.csv')

pca = pd.read_table('Data/QC/White_no_swedd.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")

info_pca['APPRDX2'] = apprdx
info_pca

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2
0,3000,3000,0.026492,0.000678,0.035338,0.064825,0.000766,-0.041875,0.033775,0.013332,...,31.0,NaN,NaN,NaN,NaN,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC
1,3001,3001,0.017766,0.017305,0.000688,-0.045234,-0.037821,-0.066630,0.010062,-0.054224,...,60.0,NaN,NaN,NaN,NaN,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD
2,3002,3002,-0.084263,-0.034856,-0.023204,0.006116,0.027577,0.008214,-0.006264,0.008593,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD
3,3004,3004,0.024563,0.033477,-0.003634,-0.005703,-0.042231,-0.059694,0.067412,0.017923,...,99.0,NaN,NaN,NaN,NaN,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC
4,3006,3006,0.008511,0.056539,0.101538,0.001351,0.006814,-0.047180,-0.012988,-0.033577,...,1.0,NaN,NaN,NaN,NaN,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,4124,4124,0.008077,0.029449,0.040843,-0.013005,0.074609,0.068977,0.010669,0.081488,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD
432,4125,4125,-0.019947,0.029816,-0.017732,-0.017475,-0.082603,0.015329,0.025788,0.053534,...,1.0,NaN,NaN,NaN,NaN,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD
433,4126,4126,-0.009423,0.001673,-0.037664,0.060811,-0.006292,0.034639,0.107867,0.040532,...,1.0,NaN,NaN,NaN,NaN,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD
434,4136,4136,0.032763,-0.016046,-0.064461,-0.015332,0.030023,0.123020,0.032174,0.084627,...,99.0,NaN,NaN,NaN,NaN,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD


In [49]:
info_pheno = info_pca.merge(images, left_on=['FID'], right_on=['PATNO'])
info_pheno

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,APPRDX2,PATNO_y,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,0.026492,0.000678,0.035338,0.064825,0.000766,-0.041875,0.033775,0.013332,...,01/2011,2011-01-19 12:44:58.0,NaN,01/2011,HC,3000,2.99,3.43,2.328620,1.973532
1,3001,3001,0.017766,0.017305,0.000688,-0.045234,-0.037821,-0.066630,0.010062,-0.054224,...,02/2011,2011-02-10 10:43:30.0,NaN,03/2011,PD,3001,1.56,1.12,-0.246956,-0.796382
2,3002,3002,-0.084263,-0.034856,-0.023204,0.006116,0.027577,0.008214,-0.006264,0.008593,...,03/2011,2011-03-04 11:04:58.0,NaN,03/2011,PD,3002,2.13,2.28,0.255480,0.339528
3,3004,3004,0.024563,0.033477,-0.003634,-0.005703,-0.042231,-0.059694,0.067412,0.017923,...,03/2011,2011-03-30 11:43:13.0,NaN,03/2011,HC,3004,5.09,5.30,2.949289,2.400680
4,3006,3006,0.008511,0.056539,0.101538,0.001351,0.006814,-0.047180,-0.012988,-0.033577,...,03/2011,2011-03-31 07:21:03.0,NaN,03/2011,PD,3006,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,4124,4124,0.008077,0.029449,0.040843,-0.013005,0.074609,0.068977,0.010669,0.081488,...,02/2013,2013-02-22 12:57:46.0,NaN,02/2013,PD,4124,1.95,1.59,-0.273727,-0.053985
429,4125,4125,-0.019947,0.029816,-0.017732,-0.017475,-0.082603,0.015329,0.025788,0.053534,...,02/2013,2013-12-10 12:36:00.0,NaN,05/2013,PD,4125,2.03,2.47,0.391926,1.005487
430,4126,4126,-0.009423,0.001673,-0.037664,0.060811,-0.006292,0.034639,0.107867,0.040532,...,03/2013,2013-03-27 08:35:42.0,NaN,03/2013,PD,4126,1.54,1.88,-0.219232,-1.158726
431,4136,4136,0.032763,-0.016046,-0.064461,-0.015332,0.030023,0.123020,0.032174,0.084627,...,03/2013,2013-03-06 07:09:02.0,NaN,06/2013,PD,4136,0.68,0.57,-1.406477,-1.922162


Since we have a different of 3 patients between our data and the DATScan data, we remove these 3 subjects to have an accurata analysis.

In [ ]:
info_pheno.iloc[:,0:2].to_csv("Data/QC/IDs_White_no_swedd.txt", header=False, index=False, sep=" ")
!plink --bfile Data/QC/White_no_swedd --keep Data/QC/IDs_White_no_swedd.txt --make-bed --out Data/QC/White_2_no_swedd

**To compute a gwas analysis with plink we need to create two files, one with the phenotypic(pheno) informations and one with the confounders(covariate)**

In [53]:
pheno = info_pheno[["FID","IID","APPRDX2","CAUDATE_R","CAUDATE_L","PUTAMEN_R","PUTAMEN_L"]]
pheno.to_csv('GWAS2/Phenotype.txt',sep="\t", index=False)
pheno

,FID,IID,APPRDX2,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,3000,HC,2.99,3.43,2.328620,1.973532
1,3001,3001,PD,1.56,1.12,-0.246956,-0.796382
2,3002,3002,PD,2.13,2.28,0.255480,0.339528
3,3004,3004,HC,5.09,5.30,2.949289,2.400680
4,3006,3006,PD,2.28,2.12,0.175997,-2.400680
...,...,...,...,...,...,...,...
428,4124,4124,PD,1.95,1.59,-0.273727,-0.053985
429,4125,4125,PD,2.03,2.47,0.391926,1.005487
430,4126,4126,PD,1.54,1.88,-0.219232,-1.158726
431,4136,4136,PD,0.68,0.57,-1.406477,-1.922162


In [54]:
age = list()

# calculate the age from the year of birth
for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT"])
    age.append(a)

info_pheno["AGE"] = age   

cov = info_pheno[["FID","IID","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.to_csv('GWAS2/Covariate.txt',sep="\t", index=False)
cov

,FID,IID,AGE,PC1,PC2,PC3,PC4,PC5
0,3000,3000,70,0.026492,0.000678,0.035338,0.064825,0.000766
1,3001,3001,65,0.017766,0.017305,0.000688,-0.045234,-0.037821
2,3002,3002,68,-0.084263,-0.034856,-0.023204,0.006116,0.027577
3,3004,3004,60,0.024563,0.033477,-0.003634,-0.005703,-0.042231
4,3006,3006,58,0.008511,0.056539,0.101538,0.001351,0.006814
...,...,...,...,...,...,...,...,...
428,4124,4124,71,0.008077,0.029449,0.040843,-0.013005,0.074609
429,4125,4125,64,-0.019947,0.029816,-0.017732,-0.017475,-0.082603
430,4126,4126,56,-0.009423,0.001673,-0.037664,0.060811,-0.006292
431,4136,4136,56,0.032763,-0.016046,-0.064461,-0.015332,0.030023


In [55]:
col = list(pheno.columns.values)[3:7] 
col

['CAUDATE_R', 'CAUDATE_L', 'PUTAMEN_R', 'PUTAMEN_L']

Core part of the analysis, the following cell create a linear model using plink. The command above can be lunched also with *--pheno-all*.

In [ ]:
for i in col:
    !plink --bfile DATA/QC/White_2_no_swedd --pheno GWAS2/Phenotype.txt --covar GWAS2/Covariate.txt --pheno-name {i} --covar-name AGE,PC1-PC5 --linear sex --adjust --out GWAS2/{i}

We select only the ADD (additive) test because is the one associated to a gwas analysis.

In [57]:
for i in col:
    gwas = pd.read_table("GWAS2/"+i+".assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/GWAS_DATScan_'+i+'.csv',index=False)

The following script is used to generate the manhatann plot and the qq plot in one single image. To set your own path and other different information you can change it. If you want you can call R code inside this python notebook using the cell magic function **%%R** at the top of a cell after loading a python module: **%load_ext rpy2.ipython**

In [ ]:
!Rscript qqman.R

## GWAS using MRI data

We load the MRI information

In [ ]:
MRI = pd.read_csv("../Processed_Images/PPMIMERGE_20200419.csv")
MRI_unique = MRI.groupby('PATNO').first().reset_index()

In [ ]:
pca = pd.read_table('Data/QC/White_2_no_swedd.eigenvec', sep=' ', header=None)
cnames = ['FID','IID']
for i in range(1,11):
    cnames.append("PC"+str(i))
pca.columns = cnames

info_pca = pca.merge(ppmi_info, left_on=['FID'], right_on=['PATNO'])

apprdx = list()

for i in info_pca['APPRDX'].to_list():
    if(i==1):
        apprdx.append("PD")
    if(i==2):
        apprdx.append("HC")

info_pca['APPRDX2'] = apprdx

info_pheno = info_pca.merge(MRI_unique, left_on=['FID'], right_on=['PATNO'])
age = list()
for i in range(len(info_pheno)):
    mm,yy = info_pheno.loc[i,"SITE_APRV"].split("/")
    a = int(yy) - int(info_pheno.loc[i,"BIRTHDT_x"])
    age.append(a)

info_pheno["AGE"] = age

In [ ]:
pheno = info_pheno[["FID","IID","APPRDX2","rh_parahippocampal_area","lh_parahippocampal_area","rh_parahippocampal_volume","lh_parahippocampal_volume"]]

**N.B.**
The phenotype related to the MRI data are quite sensitive to outliers. Outliers can be easily produced during processing of the MRI. We want to convert the raw measures into something more robust (in a way we also have to exclude subjects). A solution is to **winsorize** the data.

In [ ]:
for i in range(3,7):
       pheno.iloc[:,i] = winsorize(pheno.iloc[:,i], limits=[0.05, 0.05])

pheno.fillna("NA",inplace=True)
pheno.to_csv('GWAS2/MRI/Phenotype_Without_SWEDD.txt',sep="\t", index=False)

In [ ]:
cov = info_pheno[["FID","IID","eTIV","AGE","PC1","PC2","PC3","PC4","PC5"]]
cov.fillna("NA",inplace=True)
cov.to_csv('GWAS2/MRI/Covariate_Without_SWEDD.txt',sep="\t", index=False)

In [ ]:
for i in col:
    !plink --bfile DATA/QC/White_2_no_swedd --pheno GWAS2/MRI/Phenotype_Without_SWEDD.txt --covar GWAS2/MRI/Covariate_Without_SWEDD.txt --pheno-name {i} --covar-name eTIV,AGE,PC1-PC5 --linear sex --adjust --out GWAS2/MRI/{i+"_Without_SWEDD"}

In [33]:
for i in col:
    gwas = pd.read_table("GWAS2/MRI/"+i+"_Without_SWEDD.assoc.linear",sep="\s+")
    gwas = gwas[gwas.TEST=="ADD"]
    gwas.to_csv('GWAS2/MRI/GWAS_MRI_'+i+'_Without_SWEDD.csv',index=False)

In [ ]:
!Rscript qqman.R